In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import pickle

# functions

In [5]:
def gen_conll(filepath, nlog=1000):
    with io.open(filepath, encoding='utf-8') as fl:
        sent_count = 0
        for line in fl:
            line = line.strip()
            toks = line.split()
            #print(toks)
            # sentence break
            if len(toks) < 1:
                if len(sent) > 0:
                    sent_count += 1
                    if sent_count % nlog == 0:
                        print('LOG: '+ str(sent_count), sent)
                    doc = sent
                    words = doc.split(' ')[:-1]
                    word_labels = cats
                    sent = ""
                    cats = []
                    yield doc, words, word_labels

            # sentence start
            elif toks[0] == u'-DOCSTART-':
                sent = ""
                cats = []
            else:
                sent += toks[0] + ' '
                cats.append(toks[-1])

In [6]:
gen = gen_conll('../../../resources/data/NER/wikigold/wikigold.conll.txt')

In [7]:
# converting data to BIO from I.
total_sents = 0
f = open('../../../resources/data/NER/wikigold/train_correct.pkl', 'wb')
for data in gen:
    total_sents += 1
    doc, words, labels = data
    if len(labels[0].split('-'))>1:
        labels[0] = 'B-'+labels[0].split('-')[1]
    for idx in range(1, len(labels)):
        if labels[idx-1] == 'O' and labels[idx].split('-')[0] == 'I':
            labels[idx] = 'B-'+ labels[idx].split('-')[1]
    rec = []
    for idx, word in enumerate(words):
        rec.append((words[idx], labels[idx]))
    pickle.dump(rec, f)
f.close()

LOG: 1000 Carter also played in Blue Indigo with LeRoi Moore , Sal Soghoian , and George Melvin . 


In [8]:
# writing the data to CoNLL format in text file.
file_to_read = '../../../resources/data/NER/wikigold/train_correct.pkl'
f = open(file_to_read, 'rb')
with io.open('../../../resources/data/NER/wikigold/data_correct.txt', 'w', encoding='utf-8') as fl:
    fl.write('-DOCSTART-\n')
    
    try:
        while True:
            value = pickle.load(f)
            #print(value)
            fl.write('\n')
            for tup in value:
                #print(tup[0], tup[1])
                fl.write(tup[0] + ' ' + tup[1] + '\n')
    except EOFError as er:
        print('input finished')
f.close()

input finished
